## 0.2 - Подготовка здорового корпоративного портфеля

**Цель**: Сформировать датасет "здоровых" клиентов, исключив клиентов, находящихся в дефолте, или дефолтные периоды, из сегментов C+ с проставлением флага дефолта в течение следующего года от даты оценки.

**Prerequisites**: 
- Данные о дефолтах клиентов, обогощенный на предыдущем шаге;
- Данные о текущем портфеле клиентов за 2018-2025 гг, имеющих кредит.

**Результат**: Датасет ***healthy_portfolio.csv*** с клиентами без активных дефолтов, содержащий целевую переменную для моделирования.

----
**Подробнее**:

В данном ноутбуке происходит формирование обучающей выборки из корпоративного портфеля. 
Ключевой этап — определение двух критических переменных:
- *target* — бинарная переменная, показывающая, попадёт ли клиент в дефолт в течение следующих 365 дней (горизонт прогнозирования)
- *survive* — флаг выживания, определяющий текущий статус клиента на дату оценки

Логика флага survive следующая:
- survive = 0 присваивается, если на дату оценки клиент находится между датой дефолта и датой восстановления (то есть в активном дефолте)
- survive = 1 означает, что клиент либо ещё не входил в дефолт, либо уже восстановился после предыдущего дефолта

Это критически важное разделение: модель должна обучаться предсказывать будущие дефолты только для "живых" компаний. Включение уже дефолтных компаний исказило бы обучение, так как их дефолт — уже свершившийся факт, а не прогноз.
Для построения модели отбираются только клиенты с survive = 1. Финальный датасет содержит уникальные пары (дата, клиент), представляющие здоровый портфель для дальнейшего моделирования.

In [1]:
import pandas as pd
import numpy as np

import yaml 

import warnings
warnings.filterwarnings("ignore")

<h3>Пути</h3>

In [2]:
with open('../CONFIGS.yaml', 'r') as file:
    CONFIG = yaml.safe_load(file)

PATHS = CONFIG['data_paths']

<h3>Константные переменные</h3>

In [3]:
filepath = {
      'preprocessed_defaults' : PATHS['risk']['defaults_preprocessed']
    , 'portfolio' : PATHS['portfolio']['raw']
    , 'healthy_portfolio' : PATHS['portfolio']['healthy']
}

C_plus_segments = CONFIG['preprocessing_params']['C_plus_segments']

check_dict = {
    'М': 'M', 
    'Н': 'H', 
    'К': 'K', 
    'С': 'C'
}

### Данные

In [4]:
df_default_episodes = pd.read_csv(filepath['preprocessed_defaults'], encoding='cp1251', sep='|')

In [5]:
df_default_episodes.shape

(297, 3)

In [6]:
for date_col in ['default_date', 'recovery_date']:
    df_default_episodes[date_col]  = pd.to_datetime(df_default_episodes[date_col], format='%Y-%m-%d')

<h3>Работа с данными</h3>

In [7]:
df_portfolio = pd.read_pickle(filepath['portfolio'])

# препроцессинг
df_portfolio.rename({  'cust_id'  : 'client_id'
                     , 'total_od' : 'debt_total'}
                    , axis=1
                    , inplace=True)
df_portfolio['segment'] = df_portfolio['segment'].apply(lambda x: str(x).upper().strip())
df_portfolio = df_portfolio.replace({'segment': check_dict})

# closes_next_month_date or rounded_date
df_portfolio['date'] = pd.to_datetime(df_portfolio['report_date']) + pd.offsets.MonthBegin(1) # ближайший след. месяц

 # только те периоды, где имеется кредит
df_portfolio = df_portfolio\
                    .query('balance > 0')\
                    .sort_values(by=['client_id', 'report_date'])

In [8]:
df_portfolio.shape

(492263, 5)

In [9]:
df_segment = df_portfolio\
                .query('segment.isin(@C_plus_segments)')\
                .reset_index()\
                .rename(columns={'index': 'row_id'})\
                [['row_id', 'client_id', 'date', 'segment']]

Вот здесь сильные различия в размерности. ФИльтрация по сегменту не убирает много рекордс

In [10]:
df_segment.shape

(467721, 4)

но если сделать так (этого нет в старом ноутбуке)

In [11]:
df_segment

,row_id,client_id,date,segment
0,57678,176783,2018-02-01,C
1,160179,176783,2018-03-01,C
2,291920,176783,2018-04-01,C
3,350188,176783,2018-05-01,C
4,798078,176783,2018-06-01,C
...,...,...,...,...
467716,721205,7500722,2025-08-01,C
467717,721462,7520054,2025-08-01,C
467718,721463,7520054,2025-08-01,C
467719,721464,7520054,2025-08-01,C


In [12]:
df_segment =  df_segment\
    .drop_duplicates(subset=['client_id', 'date', 'segment'])\
    [['client_id', 'date', 'segment']] #уберу row_id пока

In [13]:
not_in_c = df_portfolio.query('~segment.isin(@C_plus_segments)').client_id.values
in_c     = df_portfolio.query('segment.isin(@C_plus_segments)').client_id.values
entered_c = set(in_c).intersection(not_in_c)

example = 706021

In [14]:
df_portfolio.query('client_id == @example')[['segment', 'report_date', 'date']].drop_duplicates()

,segment,report_date,date
731188,C,2020-01-31,2020-02-01
203452,C,2020-02-28,2020-03-01
1019333,C,2020-03-31,2020-04-01
1069984,C,2020-04-30,2020-05-01
329856,C,2020-05-29,2020-06-01
1001041,C,2020-06-30,2020-07-01
1050630,C,2020-07-30,2020-08-01
957622,C,2020-08-28,2020-09-01
635555,C,2020-09-30,2020-10-01
241908,C,2020-10-30,2020-11-01


In [15]:
df_segment.query('client_id == @example')

,client_id,date,segment
23584,706021,2020-02-01,C
23619,706021,2020-03-01,C
23655,706021,2020-04-01,C
23691,706021,2020-05-01,C
23729,706021,2020-06-01,C
23731,706021,2020-07-01,C
23738,706021,2020-08-01,C
23746,706021,2020-09-01,C
23755,706021,2020-10-01,C
23763,706021,2020-11-01,C


In [16]:
df_segment_entry_dates = df_segment\
                            .groupby(['client_id'])\
                            .agg({'date': 'min'}) \
                            .reset_index() \
                            .rename(columns={'date': 'segment_first_entry_date'})

In [17]:
df_segment_entry_dates.shape[0]

1210

In [18]:
df_segment_entry_dates\
    .groupby('client_id')['segment_first_entry_date']\
    .nunique()\
    .max()

np.int64(1)

In [19]:
df_segment_entry_dates.query('client_id == @example')

,client_id,segment_first_entry_date
47,706021,2020-02-01


In [20]:
df_within_segment_periods = pd.merge( df_portfolio,
                                      df_segment_entry_dates,
                                      how = 'inner',
                                      on = ['client_id']
                                    )

df_within_segment_periods = df_within_segment_periods\
                                .query('date >= segment_first_entry_date')\
                                .reset_index()\
                                .rename(columns={'index': 'row_id'})

In [21]:
df_within_segment_periods.shape

(491501, 7)

In [22]:
df_within_segment_periods.query('client_id == @example')

,row_id,client_id,segment,balance,report_date,date,segment_first_entry_date
23584,23584,706021,C,10000000.0,2020-01-31,2020-02-01,2020-02-01
23585,23585,706021,C,25000000.0,2020-01-31,2020-02-01,2020-02-01
23586,23586,706021,C,25700000.0,2020-01-31,2020-02-01,2020-02-01
23587,23587,706021,C,15000000.0,2020-01-31,2020-02-01,2020-02-01
23588,23588,706021,C,15000000.0,2020-01-31,2020-02-01,2020-02-01
...,...,...,...,...,...,...,...
24218,24218,706021,К2,67349819.0,2023-05-31,2023-06-01,2020-02-01
24219,24219,706021,К2,42436253.0,2023-05-31,2023-06-01,2020-02-01
24220,24220,706021,К2,77839921.0,2023-05-31,2023-06-01,2020-02-01
24221,24221,706021,К2,21000000.0,2023-05-31,2023-06-01,2020-02-01


<h4>Определение здорового портфеля</h4>

In [23]:
df_target_original = pd.merge(df_within_segment_periods,
                     df_default_episodes,
                     how = 'left',
                     on = ['client_id'])

In [24]:
df_target_original.groupby('client_id')['date'].nunique().sort_values(ascending=False)

client_id
703062     91
689330     91
713542     91
712899     91
2290511    91
           ..
7403418     1
7440765     1
7497819     1
7520054     1
7521754     1
Name: date, Length: 1210, dtype: int64

In [25]:
df_target = df_target_original.copy()

NO_DEFAULT_DATE =  pd.to_datetime('2199-01-01', format='%Y-%m-%d')
df_target.fillna({  'default_date'  : NO_DEFAULT_DATE
                  , 'recovery_date' : NO_DEFAULT_DATE
                 }, inplace=True)

for date_col in ['report_date', 'date', 'segment_first_entry_date', 'default_date', 'recovery_date']:
    df_target[date_col]  = pd.to_datetime(df_target[date_col], format='%Y-%m-%d')

default_within_year   = (df_target['default_date'] - df_target['date']).dt.days < 366
default_in_the_future =  df_target['default_date'] >= df_target['date']

df_target['target'] = np.where(default_within_year & default_in_the_future, 1, 0)

default_in_the_past    = df_target['default_date']  <= df_target['date']  
recovery_in_the_future = df_target['recovery_date'] >= df_target['date']
# находится в дефолте
df_target['survive'] = np.where(default_in_the_past & recovery_in_the_future, 0, 1)

In [26]:
df_target_aggregated = df_target\
                .groupby(['row_id'], as_index = False)\
                .agg({  'target': 'max'
                      , 'survive': 'min'}
                     )
df_target_aggregated.head()

,row_id,target,survive
0,0,0,1
1,1,0,1
2,2,0,1
3,3,0,1
4,4,0,1


In [27]:
df_target_aggregated['target'].value_counts()

target
0    457386
1     34115
Name: count, dtype: int64

In [28]:
df_target_aggregated['survive'].value_counts()

survive
1    406690
0     84811
Name: count, dtype: int64

In [32]:
healthy_portfolio = pd.merge( df_within_segment_periods,
               df_target_aggregated,
               how = 'left',
               on  = 'row_id')

healthy_portfolio.head()

,row_id,client_id,segment,balance,report_date,date,segment_first_entry_date,target,survive
0,0,176783,C,18597252.35,2018-01-31,2018-02-01,2018-02-01,0,1
1,1,176783,C,17800414.80,2018-02-28,2018-03-01,2018-02-01,0,1
2,2,176783,C,16995276.86,2018-03-30,2018-04-01,2018-02-01,0,1
3,3,176783,C,16181752.07,2018-04-28,2018-05-01,2018-02-01,0,1
4,4,176783,C,15359753.06,2018-05-31,2018-06-01,2018-02-01,0,1


In [33]:
healthy_portfolio \
    .query('survive == 0') \
    .sample(5)

,row_id,client_id,segment,balance,report_date,date,segment_first_entry_date,target,survive
140209,140209,751465,C,1.320385e+08,2019-11-29,2019-12-01,2018-02-01,1,0
63563,63563,740931,C,4.854971e+06,2023-02-28,2023-03-01,2018-02-01,0,0
192787,192788,758783,C,4.362680e+06,2025-03-31,2025-04-01,2018-02-01,0,0
72302,72302,745230,C,1.017671e+07,2020-06-30,2020-07-01,2018-02-01,0,0
11081,11081,688914,C,1.118333e+07,2022-12-30,2023-01-01,2018-02-01,0,0


In [35]:
healthy_portfolio = healthy_portfolio.query('survive == 1')
healthy_portfolio = healthy_portfolio.drop_duplicates(subset=['client_id', 'date'])
healthy_portfolio.head()

,row_id,client_id,segment,balance,report_date,date,segment_first_entry_date,target,survive
0,0,176783,C,18597252.35,2018-01-31,2018-02-01,2018-02-01,0,1
1,1,176783,C,17800414.80,2018-02-28,2018-03-01,2018-02-01,0,1
2,2,176783,C,16995276.86,2018-03-30,2018-04-01,2018-02-01,0,1
3,3,176783,C,16181752.07,2018-04-28,2018-05-01,2018-02-01,0,1
4,4,176783,C,15359753.06,2018-05-31,2018-06-01,2018-02-01,0,1


In [36]:
healthy_portfolio \
    .groupby(['date', 'client_id'], as_index = False) \
    .agg({'row_id': 'count'}) \
    .rename(columns={'row_id': 'count'}) \
    .sort_values(by=['count'], ascending = False)

,date,client_id,count
30458,2025-08-01,7521754,1
0,2018-02-01,176783,1
1,2018-02-01,197185,1
2,2018-02-01,197228,1
3,2018-02-01,678519,1
...,...,...,...
15,2018-02-01,703062,1
14,2018-02-01,702994,1
13,2018-02-01,702810,1
12,2018-02-01,702732,1


<h3>Сохранение данных</h3>

In [45]:
healthy_portfolio.query('client_id==176783')

,row_id,client_id,segment,balance,report_date,date,segment_first_entry_date,target,survive
0,0,176783,C,18597252.35,2018-01-31,2018-02-01,2018-02-01,0,1
1,1,176783,C,17800414.80,2018-02-28,2018-03-01,2018-02-01,0,1
2,2,176783,C,16995276.86,2018-03-30,2018-04-01,2018-02-01,0,1
3,3,176783,C,16181752.07,2018-04-28,2018-05-01,2018-02-01,0,1
4,4,176783,C,15359753.06,2018-05-31,2018-06-01,2018-02-01,0,1
5,5,176783,C,14529191.56,2018-06-29,2018-07-01,2018-02-01,0,1
6,6,176783,C,13689978.38,2018-07-31,2018-08-01,2018-02-01,0,1
7,7,176783,C,12842023.40,2018-08-29,2018-09-01,2018-02-01,0,1
8,8,176783,C,11985235.55,2018-09-28,2018-10-01,2018-02-01,0,1
9,9,176783,C,11119522.83,2018-10-31,2018-11-01,2018-02-01,0,1


In [49]:
healthy_portfolio.to_csv(filepath['healthy_portfolio'], index=False)